In [224]:
# score: 0.82666


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports

In [225]:
from xgboost import XGBRegressor, XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.metrics import mean_absolute_error, roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import optuna

# Get data

In [226]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

X = train_data.copy()
X.drop(['id', 'target'], axis=1, inplace=True)

y = train_data['target']

no_id_test_data = test_data.drop('id', axis=1)

display(X.head())
display(y.head())

,gravity,ph,osmo,cond,urea,calc
0,1.013,6.19,443,14.8,124,1.45
1,1.025,5.40,703,23.6,394,4.18
2,1.009,6.13,371,24.5,159,9.04
3,1.021,4.91,442,20.8,398,6.63
4,1.021,5.53,874,17.8,385,2.21


0    0
1    0
2    0
3    1
4    1
Name: target, dtype: int64

# Feature Engineering

In [227]:
display(X.columns)

def prepXy(df,X_label,y=None):
    
    data = df.drop(['gravity', 'ph', 'cond'], axis=1);
    return data, df, y
    
    selected_feat = []
    data = df.copy()
    
    # Feature Engineering
    
    
    # Ion product of calcium and urea
    data["ion_product"] = data["calc"] * data["urea"]

    # Calcium-to-urea ratio
    data["calcium_to_urea_ratio"] = data["calc"] / data["urea"]

    # Electrolyte balance
    data["electrolyte_balance"] = data["cond"] / (10 ** (-data["ph"]))

    # Osmolality-to-specific gravity ratio
    data["osmolality_to_sg_ratio"] = data["osmo"] / data["gravity"]

    # The product of osmolarity and density is created as a new property
    data['osmo_density'] = data['osmo'] * data['gravity']
    
    # ******************************************************************
    
    # Calculate ammonium concentration (assuming ammonium is not directly measured)
    data['ammonium_concentration'] = (data['gravity'] - 1.010) * (140 - (2 * data["ph"])) * 1.2

    # Calculate phosphate concentration (assuming phosphate is not directly measured)
    data['phosphate_concentration'] = (data['gravity'] - 1.010) * (32 - (0.06 * data["ph"]))

    # *****************************************************************
    
    data["gravity/ph"] = data["gravity"] / data["ph"]
    data["osmo/cond"] = data["osmo"] / data["cond"]
    data["gravity*ph"] = data["gravity"] * data["ph"]
    data["osmo*urea"] = data["osmo"] * data["urea"]
    data["cond_urea_ph"] = data["cond"] * data["urea"] / data["ph"]
    data["ph*osmo"] = data["ph"] * data["osmo"]
    data["cond*calc"] = data["cond"] * data["calc"]
    data["gravity/calc"] = data["gravity"] / data["calc"]

    # Split into features and target
    X_ret = data[X_label].values
    
    if y is None:
        y_ret = np.zeros(data.shape[0])
    else:
        y_ret = np.ravel(y)

    data.drop('ph', axis=1, inplace=True)

    
    data['target']=y_ret
    
    display(data.columns)
    
    sns.heatmap(data.corr())

    data.drop('target', axis=1, inplace=True)

    # Scaling dataset
    # scaler = MinMaxScaler()
    
    
    # scaler = StandardScaler()
    
    # X_ret = scaler.fit_transform(X_ret)

    return data,X_ret,y_ret


def feature_engineering(X):
    X.drop(['urea', 'osmo', 'ph', 'gravity'], axis=1, inplace=True)
    return X


print(X.count());

Index(['gravity', 'ph', 'osmo', 'cond', 'urea', 'calc'], dtype='object')

gravity    414
ph         414
osmo       414
cond       414
urea       414
calc       414
dtype: int64


# Remove Outliers

In [228]:
# Credit: KLYUSHNIK-ALEXSANDR

def outlier_removal(data , i):
    q1 = np.quantile(data[i] ,0.25)
    q3 = np.quantile(data[i] , 0.75)
    iqr = q3-q1
    lower_tail = q1 - 1.5*iqr
    upper_tail = q3 + 1.5*iqr
    
    
    for j in range(0, data[i].count()):
        if(data[i][j]>upper_tail or data[i][j]<lower_tail):
            data[i][j]=np.nan
            
    return data[i]

# Aggregate Features

In [229]:
def aggregate_features(X):
    
    # outlier_list = ['gravity', 'osmo', 'cond', 'urea', 'calc']
    outlier_list = ['calc', 'cond']
    for i in outlier_list:
        X[i]=outlier_removal(X ,i)
    
    my_imputer = SimpleImputer()
    X = pd.DataFrame(my_imputer.fit_transform(X))
    
    return X

# Preprocess Data

In [230]:
def preprocess_data(X, y=None):
    
    X = feature_engineering(X)
    
    X = aggregate_features(X)
    
    print('aleorcufd')
    # display(np.count_nonzero(np.isnan(X)))
    
    # display(X.columns)
    
    return X, y


'''preprocess data here'''
X, y = preprocess_data(X, y)
no_id_test_data, _ = preprocess_data(no_id_test_data)

display(X)

# cols_with_missing = [col for col in X.columns if X[col].isnull().any()]

# print(f'There are {len(cols_with_missing)} columns with missing values')

# for i in cols_with_missing:
#     print(i)
    
    
# print(X.count());



aleorcufd
aleorcufd


,0,1
0,14.8,1.45
1,23.6,4.18
2,24.5,9.04
3,20.8,6.63
4,17.8,2.21
...,...,...
409,21.4,1.53
410,19.7,0.77
411,22.2,7.68
412,12.6,1.03


# Hyperparameter Tuning

In [231]:
def optuna_study(X, y):

    def objective(trial):
        
        # For XGBRegressor
        # n_estimators = trial.suggest_int('n_estimators', 10, 1000)
        # learning_rate = trial.suggest_float('learning_rate', 0.01, 1.0)
        # max_depth = trial.suggest_int('max_depth', 1, 30)
        
        # rf = XGBClassifier(n_estimators=n_estimators,
        #                 learning_rate=learning_rate,
        #                 max_depth=max_depth)
        
        
        
        # For RandomForestClassifier
        # n_estimators = trial.suggest_int('n_estimators', 10, 1000)
        # max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 2, 100)
        # max_depth = trial.suggest_int('max_depth', 1, 30)
        
        # rf = RandomForestClassifier(n_estimators=n_estimators,
        #                 max_leaf_nodes=max_leaf_nodes,
        #                 max_depth=max_depth)
        
        
        
        # KNeighborsClassifier
        
        n_neighbors = trial.suggest_int('n_neighbors', 1, 200)
        leaf_size = trial.suggest_int('leaf_size', 1, 800)
        p = trial.suggest_int('p', 1, 3)
        
        rf = KNeighborsClassifier(n_neighbors=n_neighbors,
                                  leaf_size=leaf_size,
                                  p=p)

        score = cross_val_score(rf, X, y, n_jobs=4, cv=5, scoring='roc_auc', error_score='raise')
        
        display('score.mean(): ', score)
        return score.mean()

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=200)
    
    return study.best_params

# Study 1 (no ph + remove outlier + aggregate values, 100 trials): 0.1564147933997004
# Study 2 (no preprocessing, 100 trials): 0.12094767907088448
# Study 3 (preprocessing kudos to MOHAMMAD RAZEGHI, 200 trials): 0.18433600629551264
# Study 4 (same as 3 but max_depth allowed to be 1, 
# {'n_estimators': 31, 'learning_rate': 0.15241724886836167, 'max_depth': 1}): 0.22565032154969472

# *begin using roc_auc

# Study 5 ({'n_estimators': 109, 'learning_rate': 0.03160535667677366, 'max_depth': 1}): 0.7891310192305856
# Study 6 [I 2023-04-15 17:47:02,473] Trial 199 finished with value: 0.756868939982877 and parameters: {'n_estimators': 70, 'learning_rate': 0.21972623175664083, 'max_depth': 2}. Best is trial 196 with value: 0.7902962715736496.

print('hello')
# display(np.count_nonzero(np.isnan(X)))

best_params = optuna_study(X, y)

display(best_params)

[I 2023-04-16 12:29:35,631] A new study created in memory with name: no-name-4e5f3593-aa24-40e3-b5a5-c91f1906e327


hello


'score.mean(): '

array([0.74911868, 0.79024677, 0.83783784, 0.84488837, 0.7285628 ])

[I 2023-04-16 12:29:39,795] Trial 0 finished with value: 0.7901308917613263 and parameters: {'n_neighbors': 21, 'leaf_size': 542, 'p': 2}. Best is trial 0 with value: 0.7901308917613263.


...

[I 2023-04-16 12:29:49,702] Trial 198 finished with value: 0.7885004569787177 and parameters: {'n_neighbors': 12, 'leaf_size': 772, 'p': 1}. Best is trial 52 with value: 0.7963523305914609.


'score.mean(): '

array([0.7153349 , 0.77555817, 0.84106933, 0.8592832 , 0.71135266])

[I 2023-04-16 12:29:49,748] Trial 199 finished with value: 0.7805196500848675 and parameters: {'n_neighbors': 8, 'leaf_size': 265, 'p': 1}. Best is trial 52 with value: 0.7963523305914609.


{'n_neighbors': 15, 'leaf_size': 736, 'p': 2}

# Submission

In [232]:
# final_model = XGBRegressor(
#     n_estimators = best_params['n_estimators'], 
#     max_depth = best_params['max_depth'], 
#     learning_rate = best_params['learning_rate'],
#     n_jobs=4)

# final_model = RandomForestClassifier(
#     n_estimators = best_params['n_estimators'], 
#     max_leaf_nodes = best_params['max_leaf_nodes'], 
#     max_depth = best_params['max_depth'],
#     n_jobs=4)

final_model = KNeighborsClassifier(
    n_neighbors = best_params['n_neighbors'], 
    leaf_size = best_params['leaf_size'], 
    p = best_params['p'],
    n_jobs=4)

final_model.fit(X, y)
predictions = final_model.predict(no_id_test_data)

submission = pd.DataFrame({'id': test_data.id, 'target': predictions})

display(submission.head())

submission.to_csv(f'submission11.csv', index=False)

print('Successful')

,id,target
0,414,0
1,415,1
2,416,0
3,417,0
4,418,0


Successful
